<a href="https://colab.research.google.com/github/KKKKKIKKKK/BaekJoon.java/blob/master/2-1.Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer 실습

In [3]:
!pip install datasets

In [5]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 27.6 MB/s eta 0:00:00


#  IMDB 영화 리뷰 데이터셋을 로드하고 처리
- 목적:
IMDB 영화 리뷰 데이터셋을 로드하고, 이를 BERT 모델에 입력할 수 있는 형태로 전처리하는 단계
- 이는 감성 분석(긍정/부정 분류) 같은 자연어 처리 작업을 위한 준비

In [6]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Self-attention

이번에는 self-attention을 구현해보겠습니다.
Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

In [7]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model)

    if mask is not None:
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)
    result = self.dense(result)

    return result

# TransformerLayer를 구현

1. TransformerLayer 클래스 구조
2. Self-Attention 구현
3. Feed-Forward Network (FFN) 구현
4. Forward 패스
5. Mask 처리



In [8]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

## Positional encoding

$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

- max_len = 400, d_model = 256으로 설정하여 positional encoding을 생성


In [9]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


## Transformer 기반의 텍스트 분류기 모델을 구현
- 임베딩 층, 위치 인코딩, 여러 개의 Transformer 층, 그리고 분류 층으로 구성
- 초기화 (init 메소드)
- 순전파 (forward 메소드)
- 이 TextClassifier 모델은 Transformer 아키텍처를 기반으로 한 텍스트 분류기입니다. 입력 텍스트를 토큰화하고, 임베딩하며, 위치 정보를 추가한 후, 여러 Transformer 층을 통과시켜 특징을 추출

In [10]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, 1)

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 2, 32)

## 텍스트 분류 모델의 학습을 위한 준비 단계
- 설명:
이 코드는 텍스트 분류 모델의 학습을 위한 핵심 요소들을 설정합니다. Adam 옵티마이저를 사용하여 모델 파라미터를 효율적으로 업데이트하고, BCEWithLogitsLoss를 통해 이진 분류 문제에 적합한 손실을 계산합니다. 모델을 GPU로 이동시켜 학습 속도를 향상시키며, 0.001의 학습률로 학습을 진행할 준비를 합니다. 이러한 설정은 모델이 IMDB 리뷰 데이터셋에서 감성 분석(긍정/부정 분류)을 효과적으로 학습할 수 있도록 합니다.

In [11]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)

## 모델의 정확도를 평가하는 함수를 정의

In [12]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

## 텍스트 분류 모델의 학습 과정을 구현

In [ ]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 235.09541702270508
=========> Train acc: 0.764 | Test acc: 0.741
Epoch   1 | Train Loss: 181.78545913100243
=========> Train acc: 0.824 | Test acc: 0.781
Epoch   2 | Train Loss: 151.37181936204433
=========> Train acc: 0.861 | Test acc: 0.794
Epoch   3 | Train Loss: 130.77798080444336
=========> Train acc: 0.889 | Test acc: 0.805
Epoch   4 | Train Loss: 112.27618734538555
=========> Train acc: 0.911 | Test acc: 0.801
Epoch   5 | Train Loss: 94.61244989186525
=========> Train acc: 0.929 | Test acc: 0.808
Epoch   6 | Train Loss: 78.67021795362234
=========> Train acc: 0.944 | Test acc: 0.800
Epoch   7 | Train Loss: 64.12001316994429
=========> Train acc: 0.960 | Test acc: 0.799
Epoch   8 | Train Loss: 54.19289704039693
=========> Train acc: 0.966 | Test acc: 0.796
Epoch   9 | Train Loss: 43.67390425037593
=========> Train acc: 0.979 | Test acc: 0.800
Epoch  10 | Train Loss: 34.29198347218335
=========> Train acc: 0.980 | Test acc: 0.799
Epoch  11 | Train Loss: 29.

학습이 안정적으로 진행되며 RNN보다 빨리 수렴하는 것을 확인할 수 있습니다.
하지만 test 정확도가 RNN보다 낮은 것을 보았을 때, overfitting에 취약하다는 것을 알 수 있습니다.